<a href="https://colab.research.google.com/github/mehrdad117/Brain-Age-prediction-/blob/main/model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tpot
from sklearn.metrics import accuracy_score

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.datasets import load_boston
 
 
import pandas as pd #this is how we usually import pandas
import numpy as np #this is how we usually import numpy
 
import matplotlib #only needed to determine Matplotlib version number
#import tables # pytables is needed to read and write hdf5 files
#import openpyxl # is used to read and write MS Excel files
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
 
import xgboost
 
import warnings
warnings.filterwarnings('ignore')

**loading data**


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Finalfinal.csv')

data = data.drop('SubjectID' , axis=1)


#if data contains sex feature
#size_mapping = {
     #   #   'M': 0,
       #    'F': 1,
        #         }
#data['Sex'] = data['Sex'].map(size_mapping)


features_all = data.drop('Age', axis=1)
target = data['Age']
target = target.values.reshape(-1,1)

In [ ]:
features_all.shape , target.shape

# **split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_all, target,
                                                    train_size=0.9, test_size=0.1, random_state=42)

paste selected model here

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import OneHotEncoder, StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.preprocessing import FunctionTransformer
from copy import copy

model = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        StackingEstimator(estimator=make_pipeline(
            StackingEstimator(estimator=AdaBoostRegressor(learning_rate=0.5, loss="square", n_estimators=100)),
            ElasticNetCV(l1_ratio=0.7000000000000001, tol=0.1)
        ))
    ),
    StackingEstimator(estimator=ElasticNetCV(l1_ratio=0.15000000000000002, tol=0.1)),
    StackingEstimator(estimator=LassoLarsCV(normalize=False)),
    StackingEstimator(estimator=LassoLarsCV(normalize=False)),
    OneHotEncoder(minimum_fraction=0.1, sparse=False, threshold=10),
    VarianceThreshold(threshold=0.001),
    StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.75, learning_rate=0.5, loss="quantile", max_depth=4, max_features=0.6500000000000001, min_samples_leaf=13, min_samples_split=2, n_estimators=100, subsample=0.05)),
    LassoLarsCV(normalize=True)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)


fitting model 

In [ ]:
model.fit(X_train, y_train)

**calculate MAE for train and test dataset to calculate overfitting / underfitting**

In [ ]:
def calc_train_error(X_train, y_train, model):
    '''returns in-sample error for already fit model.'''
    predictions = model.predict(X_train)
    mae = mean_absolute_error(y_train, predictions)
    return mae
    
def calc_validation_error(X_test, y_test, model):
    '''returns out-of-sample error for already fit model.'''
    predictions1 = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions1)
    return mae
    
def calc_metrics(X_train, y_train, X_test, y_test, model):
    '''fits model and returns the RMSE for in-sample error and out-of-sample error'''
    model.fit(X_train, y_train)
    train_error = calc_train_error(X_train, y_train, model)
    validation_error = calc_validation_error(X_test, y_test, model)
    return train_error, validation_error


train_error, test_error = calc_metrics(X_train, y_train, X_test, y_test, model)
train_error, test_error = round(train_error, 3), round(test_error, 3)

print('train error: {} | test error: {}'.format(train_error, test_error))
print('train/test: {}'.format(round(test_error/train_error, 1)))

# **Cross Validation**

In [ ]:
import pandas as pd
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import LeaveOneOut

# prepare the cross-validation procedure

# two options for cv :

cv1 = KFold(n_splits=10, random_state=42, shuffle=True)
#LeaveOneOut()


# evaluate model
scores = cross_val_score(model, features_all, target, cv=LeaveOneOut() ,scoring='neg_mean_absolute_error', n_jobs=-1)


cv_predicts_n_folds = cross_val_predict(model, features_all, target, cv=10)

print("Accuracy score in each iteration: {}".format(scores))
print("Predicted class for each record: {}".format(cv_predicts_n_folds))
print("K-Fold Score: {}".format(np.mean(scores)))

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# **learning curve**

# #1

In [ ]:
import numpy as np
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt



train_sizes, train_scores, valid_scores = learning_curve(model, features_all, target, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1, shuffle=True, random_state=42)



train_scores_mean = -train_scores.mean(axis = 1)
validation_scores_mean = -valid_scores.mean(axis = 1 )
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))



### Plotting the two learning curves ###

plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training error')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation error')
plt.ylabel('MSE', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a linear regression model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,40)


# #2

In [ ]:

# Create means and standard deviations of training set scores
train_mean = -np.mean(train_scores, axis=1)
train_std = -np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = -np.mean(valid_scores, axis=1)
test_std = -np.std(valid_scores, axis=1)

# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()